# RVEA


## 约束优化

In [1]:
import numpy as np

计算nichecout
$$sh(x_1,x_2) = 1 - \frac {d(x_1,x_2)} {\sigma}$$
$ d(x_1,x_2)$表示两点间的距离

In [2]:
def caculate_nichecount(inds,sigma):
    '''
    describe：
        caculate_nichecount.
        nc(x|U)
    Args:
        inds: np.array, Nxgens
        sigma:float, radius
    return:
        nc: list , nichecout list
    '''
    N,_ = inds.shape
    nc = []
    for i in range(N):
        # caculate distance
        # if dst >\sigma,then sh(x_1,x_2) = 0; => \frac {d(\vec x_1,\vec x_2)} {\sigma} \geq 1 =>sh<0
        # we can assume:sh(x_1,x_1) = 0 
        dst = np.sqrt(
            np.sum(
               np.square (inds[i] - inds),
               axis = 1
            )
        )
        print(dst)

        sh = 1 - 1.0*dst/sigma
        
        sh[sh <0] = 0
        nc.append(np.sum(sh)-1)
    return nc

In [3]:
x = np.arange(24,dtype = float).reshape(8,3)
caculate_nichecount(x,8.0)

[  0.           5.19615242  10.39230485  15.58845727  20.78460969
  25.98076211  31.17691454  36.37306696]
[  5.19615242   0.           5.19615242  10.39230485  15.58845727
  20.78460969  25.98076211  31.17691454]
[ 10.39230485   5.19615242   0.           5.19615242  10.39230485
  15.58845727  20.78460969  25.98076211]
[ 15.58845727  10.39230485   5.19615242   0.           5.19615242
  10.39230485  15.58845727  20.78460969]
[ 20.78460969  15.58845727  10.39230485   5.19615242   0.           5.19615242
  10.39230485  15.58845727]
[ 25.98076211  20.78460969  15.58845727  10.39230485   5.19615242   0.
   5.19615242  10.39230485]
[ 31.17691454  25.98076211  20.78460969  15.58845727  10.39230485
   5.19615242   0.           5.19615242]
[ 36.37306696  31.17691454  25.98076211  20.78460969  15.58845727
  10.39230485   5.19615242   0.        ]


[0.350480947161671,
 0.70096189432334199,
 0.70096189432334199,
 0.70096189432334199,
 0.70096189432334199,
 0.70096189432334199,
 0.70096189432334199,
 0.350480947161671]

迭代更新$\sigma \epsilon$
$$\epsilon^{(s)}_i = A_i e^{-(\frac{s}{B_i})^{cp}}-\delta\\
\sigma^{(s)} =Ce^{(\frac {s}{D})^{cp}}-\delta$$

* 假设A，B，C，D都求出来了


In [4]:
z=1e-8
delta=1e-8

In [5]:
def reduce_radius(k,C,D):
    '''
    descibe:
        reduce the sigma echo iter
        $$\sigma = Ce^-(s/D)^cp -\delta$$
    Args:
        k:int the state
        C:float,a param
        D:float,a param
    return:
        R:float,the next sigma
    '''
    cp=2
    R =C*np.exp(-(k/D)**cp)-delta

    return R

In [6]:
reduce_radius(10,5,3)

7.4716692623907129e-05

In [7]:
def get_iter_params(MaxK,lowwer,N,upper,pointAmount):
    '''
    describe:
        get initial sigma and param C,D
    Args:
        MaxK:int, the max iter
        lowwer:np.array
        upper:np.array
        N:int,the dimension ,the gens count
        PointAmount:int
    return:
        R0:float ,initial sigma
        C:float,param C
        D:float,param D
    '''
    cp = 2 # constant
    difference = upper - lowwer
    production = np.prod(difference)
    R0 = 0.5*((2*N*production)/(pointAmount*np.pi))**(1.0/N) #
    C = R0+delta                                            # $C=\sigma^{(0)}+\delta
    #caculate D
    q = np.log((R0+delta)/delta)
    D = MaxK/q**(1.0/cp)
    return R0,C,D

边界范围$\epsilon$收缩


In [11]:
def reduce_boundary(k,A,B): 
    '''
    descibe:
        reduce the epsilon echo iter
        $$\epsilon = Ae^-(s/B)^cp -\delta$$
    Args:
        k:int the state
        A:np.arrray,a param
        B:np.array,a param
    return:
        e:np.array,the next epsilon
    '''
    cp=2
    e =A*np.exp(-(k/B)**cp)-delta

    return e

## 违约目标

In [22]:
def caculate_violation_objective(pop):
    '''
    describe:
        caculate the cv。$cv(\vec x) = \frac {1}{m}\sum \frac {G_i(\vec x)}{max G_i(\vec x)}
    Args:
        pop:dict 
            pop['violations'] violation value, maybe a list?
            pop['violation_objectives'] ,cv violation_objective,a float

    return:
        None
    '''
    # transform list as array
    # Dimension: N x constraint_cnt
    violations_vec  = np.array([ind['violations'] for ind in pop])
    N,constraint_cnt = violations_vec.shape
    maxG = np.max(violations_vec,axis = 0).reshape(1,-1) # for some fuck case
    cv= violations_vec/maxG  # numpy feature
    cv = np.sum(cv,axis =1)/constraint_cnt

    # transform numpy array to list
    for i in range(N):
        pop[i]['violation_objects'] = cv[i]
        
        

In [47]:
pop = []
for i in range(100):
    value = 10.0+50*np.random.rand(1,10)
    inds={'violations':list(value[0])}
    pop.append(inds)
    
    

In [48]:
caculate_violation_objective(pop)

In [49]:
pop

[{'violation_objects': 0.72686957850902068,
  'violations': [58.325237978875322,
   49.697763744129539,
   54.856524747572941,
   48.907111179873304,
   35.167875145035552,
   29.897849764402022,
   25.914191619488335,
   50.356536747947935,
   28.631983855113333,
   51.160738340283757]},
 {'violation_objects': 0.59428836594300183,
  'violations': [44.275115074118453,
   19.003603543620152,
   42.811845803752639,
   26.035483826177995,
   34.500446864956629,
   14.956954127103668,
   53.726999225933156,
   43.747657896684352,
   19.69700573219297,
   54.623624329295502]},
 {'violation_objects': 0.64819847787090434,
  'violations': [39.451438257577379,
   43.387187914082979,
   42.860627910674083,
   58.697502252696296,
   53.721385469588206,
   43.158725740228306,
   13.790627066477287,
   22.475435680278125,
   52.440957155303899,
   16.335019131957353]},
 {'violation_objects': 0.65695785270665907,
  'violations': [42.422287902751705,
   39.311610510193532,
   47.380033803735984,
   5

In [54]:
def get_vilation_param(pop,MaxK):
    '''
    describe:
        caculte the max vilotion of each constrait and initial param A  and B
    Args:
        pop:dict 
            pop['violations'] violation value, maybe a list?
        MaxK:the max iter times
    return:
        epsilon : np.array,initial epsilon
        A : np.array,param
        B : np.array,param
    '''
    cp = 2
    # transform list as array
    # Dimension: N x constraint_cnt
    violations_vec  = np.array([ind['violations'] for ind in pop])
    N,constraint_cnt = violations_vec.shape
    epsilon = np.max(violations_vec,axis = 0).reshape(1,-1) # for some fuck case
    A = epsilon + delta
    q = np.log((epsilon+delta)/delta)
    B = MaxK/q**(1.0/cp)
    return epsilon,A,B




In [56]:
epsilon,A,B = get_vilation_param(pop,100)

In [68]:
epsilon.reshape(-1,)

array([ 59.13258717,  59.59849219,  59.53292713,  59.59858956,
        59.85116186,  59.47536898,  58.32816008,  59.88756742,
        59.96632458,  59.94001349])